# Content based recommendation engine

Wondered how Google comes up with movies that are similar to the ones you like? In this section we will build one such recommendation system ourselves.

## 1. Finding the similarity

We know that our recommendation engine will be content based. So, we need to find similar movies to a given movie and then recommend those similar movies to the user. The logic is pretty straightforward. Right?

But, wait…. How can we find out which movies are similar to the given movie in the first place? How can we find out how much similar (or dissimilar) two movies are?

Let us start with something simple and easy to understand. Suppose, you are given the following two texts:

- Text A: London Paris London
- Text B: Paris Paris London

How would you find the similarity between Text A and Text B? Let’s analyze these texts...

- Text A contains the word "London" twice and the word "Paris" once.
- Text B contains the word "London" once and the word "Paris" twice.

What will happen if we try to represent these two texts in a 2D plane (with "London" in X axis and "Paris" in Y axis)? Let’s try to do this. It will look like this:

<img src="./resources/cos1.png"  style="height: 250px"/>

Here, the red vector represents "Text A" and the blue vector represents "Text B". Now we have graphically represented these two texts. So can we find out the similarity between these two texts?

The answer is "Yes, we can". But, exactly how? These two texts are represented as vectors. Right? So, we can say that two vectors are similar if the distance between them is small. By distance, we mean the angular distance between two vectors, which is represented by θ (theta).

<img src="./resources/cos2.png"  style="height: 250px"/>

By thinking further from the machine learning perspective, we can understand that the value of cos θ makes more sense to us rather than the value of θ (theta) because, the cosine function will map the value of θ in the first quadrant between 0 to 1 (Remember? cos 90° = 0 and cos 0° = 1 ). Can you see that the smaller θ is (more similarity between the two texts), the larger the value of cos θ will be?
 
<img src="./resources/cos3.png"  style="height: 250px"/>

## 2. Cosine_similarity()

Don’t get scared, we don’t need to implement the formula from scratch to calculate cos θ. We have our friend Scikit-Learn to calculate that for us :) Just remember: cosine similarity is a simple, but very effective vector similarity metric, that's heavily used, especially in the field of NLP (Natural Language Processing).

Let’s see how we can do that.

At first, we need to have text A and B in our program (we will take four cities):

In [1]:
text = ["London Paris London Amsterdam", "Paris Paris London Brussels"]

Now, we need to find a way to represent these texts as vectors, because we need to calculate with them. So, we have to kind of 'encode' our text, to get our numerical representation, in the form of a vector (not just a single scalar, so we have more expressive power).  

The CountVectorizer() class can do this for us.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
count_matrix = cv.fit_transform(text)

count_matrix gives us a sparse matrix. 

In [3]:
count_matrix

<2x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

To make it in human readable form, we need to apply the `toarrray()` method over it. And before printing out this count_matrix, let us first print out the feature list (or, word list), which has been fed to our `CountVectorizer()` object.

In [5]:
print(cv.get_feature_names_out())
print(count_matrix.toarray())

['amsterdam' 'brussels' 'london' 'paris']
[[1 0 2 1]
 [0 1 1 2]]


This indicates that the word amsterdam occurs 1 time in A and 0 times in B, brussels occurs 0 times in A and 1 time in B, london occurs 2 times in A and 1 time in B and paris occurs 1 time in A and 2 times in B. Makes sense. Right?

Now, we need to find cosine  similarity between these vectors to find out how similar they are to each other. We can calculate this using `cosine_similarity()` function:

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(count_matrix)
print(similarity_scores)

[[1.         0.66666667]
 [0.66666667 1.        ]]


What does this output indicate? We can interpret this output like this: 

1. Each row of the similarity matrix indicates each sentence of our input. So, row 0 = Text A and row 1 = Text B.
2. The same thing applies for columns.

To get a better understanding of this, we can say that the output given above is the same as the following:

```
        Text A      Text B
Text A    1          0.66
Text B   0.66         1
```

By interpreting this, the output says that Text A is similar to Text A (itself) by 100% (position [0,0]) and Text A is similar to Text B by 66% (position [0,1]). We can easily see that the output is always going to be a symmetric matrix. Because if Text A is similar to Text B by 66% then, Text B is also going to be similar to Text A by 66%.

## 3. Building the recommendation engine - Exercise

Now we know how to find similarity between contents. So, let’s try to apply this knowledge to build a content based movie recommendation engine. 

### 3.1 Read the data

Import all the required libraries and then read the `movie_dataset.csv` file from the `resources` directory using the `read_csv()` method.

Explore the data by showing the first 5 lines.

You will see that it has many extra info about a movie. We don’t need all of them. So, we choose the keywords, cast, genres and director column to use as our feature set. We also need to clean and preprocess the data for our use. We will fill all the NaN values with blank strings in the dataframe (for the columns keywords, cast, genres and director).

In [6]:
features = ['keywords', 'cast', 'genres', 'director']

# filling all NaNs with blank strings
for feature in features:
    movies_df[feature] = movies_df[feature].fillna('') 

NameError: name 'movies_df' is not defined

Your next task is to create a function for combining the values of these columns into a single string and call this function over each row of our dataframe. The name of the new column is `combined_features`.

Check the content of the new `combined_features` column by showing the first 5 rows.

### 3.2 Create the matrix

Now that you have obtained the combined_features column, you can now feed these strings to a CountVectorizer() object for getting the count matrix.

Print the total number of different words found in the combined_features column.

Print all the words found in the combined_features column.

Print the number of films and the number of features.

Print the number of films and the number of words.

Print the content of the count_matrix.

Now, obtain the cosine similarity matrix from the count matrix.

We will define two helper functions to get movie title from movie index and vice-versa.

In [ ]:
def get_title_from_index(index):
    return movies_df[movies_df.index == index]["title"].values[0]
def get_index_from_title(title):
    return movies_df[movies_df.title == title]["index"].values[0]

Print the index of Interstellar.

Print the title of the film with index 95.

### 3.3 Find similar movies

We will need the functions `enumerate()` and `list()`:

In [ ]:
x = ['apple', 'banana', 'cherry']
# enumerate over x and make a tuple (index, fruit)
print(list(enumerate(x)))

- We will find the index of the movie the user likes. 
- After that, we will access the row corresponding to this movie in the similarity matrix. Thus, we will get the similarity scores of all other movies from the current movie.
- Then we will enumerate through all the similarity scores of that movie to make a tuple of movie index and similarity score. 

In [ ]:
movie_user_likes = "Pulp Fiction"
movie_index = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movie_index]))

This will convert a row of similarity scores like this [1 0.5 0.2 0.9] to this [(0, 1) (1, 0.5) (2, 0.2) (3, 0.9)]. Here, each item is in this form (movie index, similarity score).

In [ ]:
print(similar_movies)

### Question

If you print the index of Pulp Fiction, what will be the value of y in (index, y) in similar_movies? Makes sense? Right?

In [ ]:
print(movie_index)

### 3.4 Sort and print Top 9 similar movies

We will sort the list similar_movies according to similarity scores in descending order. Since the most similar movie to a given movie will be itself, we will discard the first element after sorting the movies.

In [ ]:
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

In [ ]:
i = 0
print("Top 9 similar movies to " + movie_user_likes + " are:\n")
for element in sorted_similar_movies:
    print(get_title_from_index(element[0]))
    i = i + 1
    if i > 8:
        break

You can compare your results with the collaborative filtering based recommendation engine from Google. 4/9. Mmm, not bad.

<img src="./resources/pulp.png"/>

Maybe you can try to find similar movies for some other movie you like.